<a href="https://colab.research.google.com/github/Ayushsunny/First-Aid-Recommendation-Bot/blob/main/First_Aid_Recommendation_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\singh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
import gtts
from io import BytesIO
import speech_recognition as sr
import numpy
import tflearn
import tensorflow
import random
import json
import pickle
from playsound import playsound

Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)
Scipy not supported!


In [3]:
with open("intents.json") as file:
	data = json.load(file)

In [4]:
words = []
labels = []
docs_x = []
docs_y = []

for intent in data["intents"]:
	for pattern in intent["patterns"]:
		wrds = nltk.word_tokenize(pattern)
		words.extend(wrds)
		docs_x.append(wrds)
		docs_y.append(intent["tag"])

	if intent["tag"] not in labels:
		labels.append(intent["tag"])

words = [stemmer.stem(w.lower()) for w in words if w != "?"]
words = sorted(list(set(words)))

labels = sorted(labels)


training = []
output = []

out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
	bag = []

	wrds = [stemmer.stem(w) for w in doc]

	for w in words:
		if w in wrds:
			bag.append(1)
		else:
			bag.append(0)
	output_row = out_empty[:]
	output_row[labels.index(docs_y[x])] = 1

	training.append(bag)
	output.append(output_row)

training = numpy.array(training)
output = numpy.array(output)

In [5]:
tensorflow.compat.v1.reset_default_graph()

net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [6]:
try:
	model.load("model.tflearn")
except:
	model = tflearn.DNN(net)
	model.fit(training, output, n_epoch=300, batch_size=8, show_metric=True)
	model.save("model.tflearn")

INFO:tensorflow:Restoring parameters from C:\Users\singh\PycharmProjects\First-Aid-Recommendation-Bot\model.tflearn


In [7]:
def bag_of_words(s,words):
	bag = [0 for _ in range(len(words))]


	s_words = nltk.word_tokenize(s)
	s_words = [stemmer.stem(word.lower()) for word in s_words]

	for se in s_words:
		for i, w in enumerate(words):
			if w == se:
				bag[i] = 1

	return numpy.array(bag)

In [8]:
def chat(request):

	inp = request

	results = model.predict([bag_of_words(inp,words)])[0]
	results_index = numpy.argmax(results)
	tag = labels[results_index]

	if results[results_index] > 0.5:
		for tg in data["intents"]:
			if tg['tag'] == tag:
				responses = tg['responses']

		tts = gtts.gTTS(random.choice(responses), tld="co.uk")
		tts.save("temp.mp3")

		playsound("temp.mp3")

		print(random.choice(responses))
		print("\n")

	else:
		print("I didnt get that, try again")

In [9]:
r = sr.Recognizer()
mic = sr.Microphone(device_index=0)

In [11]:
def initialize_chat():
	print("Start Talking with the bot")
	while True:
		print("Ask me anything: ")
		with mic as source:
			r.adjust_for_ambient_noise(source)
			audio = r.listen(source)
		result = r.recognize_google(audio)
		if result == "quit" or result == "exit" or result == "stop":
			print("Thanks for using the bot, exiting....")
			break
		chat(result)

In [12]:
initialize_chat()

Start Talking with the bot
Ask me anything: 
Wash the cut properly to prevent infection and stop the bleeding by applying pressure for 1-2minutes until bleeding stops. Apply Petroleum Jelly to make sure that the wound is moist for quick healing. Finally cover the cut with a sterile bandage. Pain relievers such as acetaminophen can be applied.


Ask me anything: 
Thanks for using the bot, exiting....
